# Calculate the correlations for each layer and create input RDMs

## Set up the environment

In [ ]:
import numpy as np
import os
import json
import h5py
import tables

## Define functions to create filenames

In [ ]:
def getFileName(n_samples, name):
    return name \
        + "_{}_".format(n_samples) \
        + "_{}_".format(model_name) \
        + "_{}".format(layer_name)  \
        + ".npy"
#         + datetime.datetime.now().replace(microsecond=0).isoformat() \
        

In [ ]:
def getFileNameh5(n_samples, name):
    return name \
        + "_{}_".format(n_samples) \
        + "_{}_".format(model_name) \
        + "_{}".format(layer_name)  \
        + ".h5"
#         + datetime.datetime.now().replace(microsecond=0).isoformat() \
        

## Define the correlation function

In [ ]:
# calculate INPUT RDM optimized
def correlationd_matrix(batch_size): #(list_of_activations, n) ,array_activations
    #turn into h5py
    #flattened = np.array(activations).reshape(NR_OF_SAMPLES,-1) #if needed
#     print("Saving into .h5 file")
#     file_name = os.path.join(ROOT_PATH+'activations/',getFileNameh5(NR_OF_SAMPLES,'activations'))
#     h5f = h5py.File(file_name, 'w')
#     h5f.create_dataset('flattened',dtype='f') #, data=flattened
#     h5f.close()
#     print("Receiving activation length")

    file_name = os.path.join(ROOT_PATH+'activations/',getFileNameh5(NR_OF_SAMPLES,'activations'))
#     h5f = h5py.File(file_name, 'r') ?? check if needed
    g = tables.open_file(file_name, mode='r') #create tables
    total_size, activation_length = np.array(g.root.flattened).shape
    print("Activation Length: ", activation_length)
    print("Total Size: ", total_size)
    g.close()
    
#     def pearsonr_optimized(xm, ss_xm, ym, ss_ym):
# #         x = np.asarray(x)
# #         y = np.asarray(y)
# #         n = len(x)
# #         mx = x.mean()
# #         my = y.mean()
# #         xm, ym = x - mx, y - my
#         r_num = np.add.reduce(xm * ym)
#         r_den = np.sqrt(ss_xm * ss_ym)
#         r = r_num / r_den

#         # Presumably, if abs(r) > 1, then it is only some small artifact of floating
#         # point arithmetic.
#         r = max(min(r, 1.0), -1.0)

#         return r

#     #copied directly from scipy sources without change 
#     def ss(a, axis=0):
#         def _chk_asarray(a, axis):
#             if axis is None:
#                 a = np.ravel(a)
#                 outaxis = 0
#             else:
#                 a = np.asarray(a)
#                 outaxis = axis

#             if a.ndim == 0:
#                 a = np.atleast_1d(a)

#             return a, outaxis

#         a, axis = _chk_asarray(a, axis)
#         return np.sum(a*a, axis)

    #memmap is used to access a part of the file. maybe: use memmap in the loop to access the first 1000 samples?    
    act = np.memmap(file_name, mode="r", shape=(total_size, activation_length)) #numsamples x numfeatures
    correlationd = np.zeros((total_size,total_size))
    correlationd[:] = np.nan
    total = sum(x+1 for x in range(int(total_size / batch_size))) #num 1000-wise comparisons to do: 55
    index = 0
    
#     # pre-allocate necessary values for the correlation
#     centered_activations = np.ones(list_of_activations.shape) #num samples x num features
#     centered_activations[:] = np.nan

#     centered_squared_summed_activations = np.ones((list_of_activations.shape[0],)) #num samples x 1
#     centered_squared_summed_activations[:] = np.nan
    
#     #obtain the squared sum of activations
#     for i in range(n):      
#         centered_activations[i] = list_of_activations[i] - list_of_activations[i].mean()    
#         centered_squared_summed_activations[i] = ss(centered_activations[i]) 
    
#     #pre-allocate the input rdm matrix
#     correlationd = np.empty((n,n))
#     correlationd[:] = np.nan
    
#     #correlate between each pair of samples
#     for i in range(n):
#         for j in range(i + 1, n):
#             correlationd[i,j] = correlationd[j, i] = 1 - pearsonr_optimized(centered_activations[i], centered_squared_summed_activations[i], centered_activations[j], centered_squared_summed_activations[j])
      
#     return(correlationd)

   
    for i in range(int(total_size / batch_size)):
       
        start_1 = batch_size*i
        end_1 = batch_size*(i+1)
        
        list_of_activations_1 = act[start_1:end_1,:]
        
        for j in range(int(total_size / batch_size)-i):
            index += 1
            print("New Iteration: i = {0}, j = {1}; {2}/{3}".format(i,j,index,total))
            
            start_2 = batch_size*(i+j)
            end_2 = batch_size*(i+j+1)
            
            list_of_activations_2 = act[start_2:end_2,:]
            correlationd[start_1:end_1,start_2:end_2] = 1-np.corrcoef(np.concatenate((list_of_activations_1,list_of_activations_2),1))
            
    return(correlationd)

Load the activations and perform the correlation. Save the results into a matrix (Input RDM).

In [ ]:
# #load json file with the layers of interest (resnets)
# print('Loading the json file')
# ROOT_PATH = '/mnt/raid/ni/agnessa/RSA/' 
# json_file_layers=os.path.join(ROOT_PATH,'resnets_selected_layers.json')
# with open(json_file_layers, "r") as fd:
#     selected_layers = json.load(fd)
# model_name, layer_name = selected_layers[16].get('model'),selected_layers[16].get('layer') #change the index at each iteration     
# NR_OF_SAMPLES = 10000 #num classes*num samples per class;

# # #load activations
# # path = os.path.join(ROOT_PATH + 'activations/', getFileName(NR_OF_SAMPLES, "activations"))

# # # save np.load
# # np_load_old = np.load

# # # modify the default parameters of np.load
# # np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
# # print('Loading activations for model ->',model_name,',layer ->',layer_name)
# # flattened = np.load(path)
# # np.load = np_load_old

# #correlation matrix = input RDM
# print('calculating correlations for model ->',model_name, 'and layer ->', layer_name)
# corr_matrix = correlationd_matrix(1000) 
# path = os.path.join(ROOT_PATH + 'Input_RDM/', getFileName(NR_OF_SAMPLES, "Input_RDM"))
# print("Save Input RDM -> {}".format(path))
# np.save(path, np.array(corr_matrix)) 

In [ ]:
import numpy as np
import os
import json
#load json file with the layers of interest (resnets)
print('Loading the json file')
ROOT_PATH = '/mnt/raid/ni/agnessa/RSA/' 
json_file_layers=os.path.join(ROOT_PATH,'resnets_selected_layers.json')
with open(json_file_layers, "r") as fd:
    selected_layers = json.load(fd)
model_name, layer_name = selected_layers[16].get('model'),selected_layers[16].get('layer') #change the index at each iteration     
print('Model ->',model_name,', layer ->', layer_name)
NR_OF_SAMPLES = 10000 #num classes*num samples per class;


In [ ]:
print('calculating correlations for model ->',model_name, 'and layer ->', layer_name)
corr_matrix = correlationd_matrix(1000) 
path = os.path.join(ROOT_PATH + 'Input_RDM/', getFileName(NR_OF_SAMPLES, "Input_RDM"))
print("Save Input RDM -> {}".format(path))
np.save(path, np.array(corr_matrix)) 

In [ ]:
print("Saving into .h5 file")
file_name = os.path.join(ROOT_PATH+'activations/',getFileNameh5(NR_OF_SAMPLES,'activations'))
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('flattened',data=flattened) 
h5f.close()

In [ ]:
#create fake h5py file

file_name_fake = os.path.join(ROOT_PATH+'activations/',getFileNameh5(NR_OF_SAMPLES,'fake'))
hf = h5py.File(file_name_fake, 'w')
arr = np.arange(100)
hf.create_dataset('fake_dataset',data=arr)
# # list(h5f.keys())
# fl = hf.get('flattened') 
# print(flattened.shape)
# # flattened = dset.read_direct(fl)
# # np.array(fl)
# # flattened = np.array(fl)
# # # ['flattened'][()]
# # type(fl)
# # total_size, activation_length = flattened.shape

In [ ]:
#open fake dataset
file_name_fake = os.path.join(ROOT_PATH+'activations/',getFileNameh5(NR_OF_SAMPLES,'fake'))
hfread = h5py.File(file_name_fake, 'r')
f = hf.get('fake_dataset') 
print(f.shape)

In [ ]:
print(file_name)

In [ ]:
#load activations
path = os.path.join(ROOT_PATH + 'activations/', getFileName(NR_OF_SAMPLES, "activations"))

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
print('Loading activations for model ->',model_name,',layer ->',layer_name)
flattened = np.load(path)
np.load = np_load_old